# Proyecto 2 - Inteligencia Artificial 2023

### Comisión 4 - Fredes Simón - Ilz Matias



---


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import warnings
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
warnings.filterwarnings("ignore")

from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import Sequential, layers, Input

# Carga y análisis de datos

In [19]:
# Cargar el dataset.
df_futbol = pd.read_csv('./football_train.csv')

print("Cantidad de ejemplos: {}\nCantidad de columnas: {}".format(df_futbol.shape[0], df_futbol.shape[1]))
print()

# Nombre de las columnas
print("Columnas del dataset: {}".format(list(df_futbol.columns)))
print()

# Ver los primeros ejemplos del dataset.
df_futbol

Cantidad de ejemplos: 1409
Cantidad de columnas: 23

Columnas del dataset: ['Team', 'Season', 'Matchday', 'Manager', 'Stadium', 'Stadium_capacity', 'Player_age_avg', 'Possession_%', 'Shots', 'Shots_on_target', 'Touches', 'Passes', 'Tackles', 'Clearances', 'Corners', 'Offsides', 'Fouls_conceded', 'Goals', 'ConcededGoals', 'Yellow_cards', 'Red_cards', 'Position', 'Field_zone']



,Team,Season,Matchday,Manager,Stadium,Stadium_capacity,Player_age_avg,Possession_%,Shots,Shots_on_target,...,Clearances,Corners,Offsides,Fouls_conceded,Goals,ConcededGoals,Yellow_cards,Red_cards,Position,Field_zone
0,Man Utd,22-23,19,Erik ten Hag,Old Trafford,74031,25.6,53.347059,244.0,93.0,...,337.0,76.0,36.0,198.0,27,20,43.0,0.0,top-4,high
1,Nott'm Forest,22-23,12,NaN,City Ground,30404,26.3,42.490909,108.0,27.0,...,238.0,35.0,15.0,127.0,7,23,30.0,0.0,media-tabla,NaN
2,Crystal Palace,21-22,2,NaN,Selhurst Park,25486,25.7,45.500000,11.0,NaN,...,44.0,5.0,2.0,23.0,0,3,3.0,0.0,media-tabla,low
3,Southampton,22-23,11,Rubén Sellés,St Mary's Stadium,32689,24.2,42.820000,115.0,37.0,...,212.0,40.0,5.0,103.0,9,18,13.0,0.0,descenso,low
4,West Ham,21-22,12,David Moyes,London Stadium,62500,27.6,49.166667,180.0,NaN,...,225.0,67.0,17.0,107.0,23,14,14.0,1.0,4-a-7,middle
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1404,Leeds,22-23,27,Jesse Marsch,Elland Road,40000,23.5,50.016000,309.0,96.0,...,424.0,136.0,41.0,312.0,29,40,50.0,2.0,descenso,NaN
1405,Man Utd,22-23,36,Erik ten Hag,Old Trafford,74031,25.6,53.551613,461.0,174.0,...,640.0,155.0,71.0,351.0,47,37,66.0,2.0,top-4,high
1406,Liverpool,22-23,2,Jurgen Klopp,Anfield,61276,25.6,69.750000,35.0,8.0,...,22.0,8.0,7.0,16.0,3,3,2.0,1.0,4-a-7,high
1407,Newcastle,22-23,4,Eddie Howe,St James' Park,52257,27.0,50.525000,60.0,23.0,...,81.0,32.0,9.0,40.0,6,4,8.0,1.0,top-4,high


# Procesamiento de los datos

In [20]:
df_futbol.info()
different_values = df_futbol['Manager'].unique()
print("Different values in column 'Manager':", different_values)

different_values = df_futbol['Team'].unique()
print("Different values in column 'Team':", different_values)


promedio_shots_on_target = df_futbol['Shots_on_target'].mean()
print("Average Shots_on_target:", promedio_shots_on_target)

#completamos los valores nulos con el promedio
df_futbol['Shots_on_target'].fillna(df_futbol['Shots_on_target'].mean(), inplace=True)

# Complete the null values in the 'Manager' column with the string 'Other'
df_futbol['Manager'].fillna('Other', inplace=True)

field_zone_values = df_futbol['Field_zone'].values
print("Values of column 'Field_zone':", field_zone_values)

#eliminamos passes?
#df_futbol.drop('Passes', axis=1, inplace=True)
df_futbol
mapeo = {
'Man Utd' : 1,
"Nott'm Forest": 2, 
'Crystal Palace':3, 
'Southampton':4, 
'West Ham':5,
'Spurs':6,
'Brighton':7,
'Norwich':8, 
'Arsenal':23, 
'Watford':9,
'Aston Villa':10, 
'Brentford' :11,
'Leicester':12,
'Fulham' :13,
'Leeds':14,
'Newcastle':15,
'Wolves':16,
'Man City':17,
'Burnley' :18,
'Bournemouth':19, 
'Everton' :20,
'Chelsea' :21,
'Liverpool':22,
}

# Modificamos la columna 'Pclass' a partir del mapeo anterior
df_futbol['Team'] = df_futbol['Team'].map(mapeo)




<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1409 entries, 0 to 1408
Data columns (total 23 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Team              1409 non-null   object 
 1   Season            1409 non-null   object 
 2   Matchday          1409 non-null   int64  
 3   Manager           942 non-null    object 
 4   Stadium           1409 non-null   object 
 5   Stadium_capacity  1409 non-null   int64  
 6   Player_age_avg    1409 non-null   float64
 7   Possession_%      1409 non-null   float64
 8   Shots             1409 non-null   float64
 9   Shots_on_target   714 non-null    float64
 10  Touches           1409 non-null   float64
 11  Passes            220 non-null    float64
 12  Tackles           1409 non-null   float64
 13  Clearances        1409 non-null   float64
 14  Corners           1409 non-null   float64
 15  Offsides          1409 non-null   float64
 16  Fouls_conceded    1409 non-null   float64


# Desarrollo de un Árbol de Decisión

In [21]:
## Valores de la columna target. (lo que se desea predecir)
Y = df_futbol['Position'].values

# Porción del dataframe con los atributos. (lo que se usara para predecir)
X = df_futbol.drop(columns = ["Position"])

# Porcentajes para datos de validación y test.
test_size = 0.15
val_size = 0.10

# La función train_test_split separa los datos (X, Y) en dos porciones
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = test_size, random_state = 10)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size = val_size, random_state = 10)

# Nos guardamos los nombres de los atributos para más adelante
feature_names = X_train.columns.values[:-1]

print("Datos de entrenamiento:\n\tCantidad de ejemplos (filas): {}\n\tCantidad de atributos (columnas): {}\n".format(X_train.shape[0], X_train.shape[1]))
print("Datos de validación:\n\tCantidad de ejemplos (filas): {}\n\tCantidad de atributos (columnas): {}\n".format(X_val.shape[0], X_val.shape[1]))
print("Datos de test:\n\tCantidad de ejemplos (filas): {}\n\tCantidad de atributos (columnas): {}".format(X_test.shape[0], X_test.shape[1]))

# Creamos un árbol de decisión (AD)
PROF = 5
clf = DecisionTreeClassifier(max_depth = PROF)

# Se ajusta el AD con los datos de entrenamiento
clf = clf.fit(X_train, y_train) # ajustamos el AD

print(f"¿Qué profundidad tiene el árbol creado?\n{clf.get_depth()}\n")

print(f"¿Cuántas hojas tiene el árbol?\n{clf.get_n_leaves()}\n")

print(f"¿Cuántos de los atributos fueron usados?\n{clf.n_features_in_}\n")

print(f"¿Cuántos atributos eran en total?\n{len(X_train.columns.values)}\n")

print(f"¿Qué porcentaje de aciertos tuve datos de verificación?\n{clf.score(X_val, y_val):.3} de accuracy (exactitud)\n")

print("Reporte de clasificación\n")
predictions = clf.predict(X_test)
print(classification_report(y_test, predictions))

Datos de entrenamiento:
	Cantidad de ejemplos (filas): 1077
	Cantidad de atributos (columnas): 22

Datos de validación:
	Cantidad de ejemplos (filas): 120
	Cantidad de atributos (columnas): 22

Datos de test:
	Cantidad de ejemplos (filas): 212
	Cantidad de atributos (columnas): 22


ValueError: could not convert string to float: '22-23'

# Desarrollo de una Red Neuronal Artificial